In [ ]:
# importing all needed libraries
import datetime
import requests
import json
import pandas as pd

In [11]:
# creating a function to make weekly dates within a certain time frame (arguments are the start and end date)
def get_weekly_dates(d_start, d_end):
    week_dates = []
    week_date = d_start
    
    while week_date <= d_end:
        week_dates.append(week_date.isoformat())
        week_date += datetime.timedelta(weeks=1)
    return week_dates

# edit here for different data range, start date should be on a saturday
d_start = datetime.date(1980, 1, 5)
d_end = datetime.date(2024, 10, 30)

week_dates = get_weekly_dates(d_start, d_end)

In [12]:
# organizing week dates into a dictionary by year
y_start = d_start.year
y_end = d_end.year

week_dictionary = {}

for w in week_dates:
    year_of_week = datetime.date.fromisoformat(w).year
    if year_of_week not in week_dictionary:
        week_dictionary[year_of_week] = [w]
    else:
        week_dictionary[year_of_week].append(w)

In [13]:
# calling on billboard hot 100 api and exporting the data in csvs for each year
# DO NOT RUN THIS BEFORE MAKING THE DATA RANGE SMALLER, API USE REQUIRED A PAID SUBSCRIPTION AND RUNNING TOO MANY CALLS PER DAY RESULTS IN ADDITIONAL FEES
for year in week_dictionary:
    year_df = pd.DataFrame()

    for week in week_dictionary[year]:
        try:
            
            bb_weekly_url = "https://billboard-api2.p.rapidapi.com/hot-100"
            params = {
                "date": week,
                "range": "1-100"
            }
            headers = {
                "x-rapidapi-key": "65155348c9mshaa21262afddd264p14c37bjsn21ec97b632c2",
                "x-rapidapi-host": "billboard-api2.p.rapidapi.com"
            }
            
            bb_weekly_response = requests.get(bb_weekly_url, headers=headers, params=params).json()
            bb_weekly_df = pd.DataFrame(bb_weekly_response["content"])
            bb_weekly_df = bb_weekly_df.transpose()
            bb_weekly_df["date"] = week
        
            year_df = pd.concat([year_df, bb_weekly_df])

        except:
                print(f"did not save chart from {week}")
    
    year_df.to_csv(f'resources/billboard_hot_100_csvs/{year}_bb_hot_100.csv', index=False)